In [2]:
import requests
import os
import csv
from urllib.parse import quote


In [3]:

# CLIENT_ID = os.getenv("ORCID_ID")
CLIENT_ID = "APP-UGRLNMYQKMKP1JMR"
# CLIENT_SECRET = os.getenv("ORCID_SECRET")
CLIENT_SECRET = "4a304a32-cf67-415e-9c41-579c3ba857a6"
print(CLIENT_ID)
print(CLIENT_SECRET)
token = requests.post("https://orcid.org/oauth/token", data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "grant_type": "client_credentials",
    "scope": "/read-public",
}, headers = {"Accept":"application/json"})

APP-UGRLNMYQKMKP1JMR
4a304a32-cf67-415e-9c41-579c3ba857a6


In [4]:
token.json()

{'access_token': '8adc47ee-8d3b-42ed-b652-03b0bad1d95f',
 'token_type': 'bearer',
 'refresh_token': '10558944-a4cc-409d-815f-f5dffb945691',
 'expires_in': 631138518,
 'scope': '/read-public',
 'orcid': None}

In [5]:
ORCID_SEARCH = f"https://api.orcid.org/v3.0/csv-search/?q=affiliation-org-name:(%22Boston%20University%22+OR+BU)&fl=orcid,given-names,family-name"

In [6]:
print(ORCID_SEARCH)
response = requests.get(
    ORCID_SEARCH, 
    headers = {
        "Accept": "application/vnd.orcid+json",
        "Authorization": f"Bearer {token.json()['access_token']}",
    }
)

https://api.orcid.org/v3.0/csv-search/?q=affiliation-org-name:(%22Boston%20University%22+OR+BU)&fl=orcid,given-names,family-name


In [7]:
print(response)
print(response.text)

<Response [403]>
{
  "error" : "access_denied",
  "error_description" : "Access is denied"
}


In [114]:
def get_conf_rankings(ranking_csv_file):
    rank_lookup = {}
    with open(ranking_csv_file, "r") as csv_fh:
        core_csv = csv.reader(csv_fh)
        for row in core_csv:
            # rank_lookup[re.search(r"\d{4}", row[3]).group(0)][row[2]]=row[4]
            rank_lookup[row[2]]=row[4]
    return rank_lookup

In [115]:
core_ranks = get_conf_rankings("CORE.csv")

In [116]:
core_ranks["WWW"]

'A*'

In [119]:
from neo4j import GraphDatabase
URI = "neo4j://localhost:7687"
def run_cypher_match(neo4j_driver, db_name, query, params=None):
    session = neo4j_driver.session(database=db_name)
    try:
        result = session.run(query, parameters=params)
        return [line for line in result]
    finally:
        session.close()

def get_confs():
    with GraphDatabase.driver(URI, auth=("neo4j", "openreview")) as driver:
        batch = run_cypher_match(
        driver,
        "open-review-data",
        f"MATCH (c:Conference) RETURN c.id AS id, c.name AS name, c.year as year",
    )
    return batch

def set_conf_rank(conf_id, rank, acr):
    props = {"acronym":acr, "rank":rank}
    query = """
    MATCH (c:Conference {id: $id})
    SET c += $newProperties
    RETURN c
    """    
    with GraphDatabase.driver(URI, auth=("neo4j", "openreview")) as driver:
        with driver.session(database="open-review-data") as session:
            session.execute_write(lambda tx: tx.run(query, id=conf_id, newProperties=props))
        
def find_rank(lookup, acr):
    try:
        return lookup[acr]
    except KeyError:
        return "Unranked"

def add_rank_to_conferences():
    confs = get_confs()
    conf_rank = get_conf_rankings("CORE.csv")
    for conf in confs:
        shortname = conf["name"].split(".")[0]
        
        if shortname=="eswc-conferences":
            shortname = "ESWC"

        elif shortname=="aclweb":
            shortname = "ACL"

        elif shortname=="sigmobile":
            shortname = "Mobisys"

        elif shortname=="humanrobotinteraction":
            shortname = "HRI"

        elif shortname=="ijcai":
            shortname = "IJCAI"

        elif shortname=="icaps-conference":
            shortname = "ICAPS"
        
        elif shortname=="ECMLPKDD" or shortname=="ECMLPKDD".lower():
            shortname = "ECML PKDD"
        
        elif shortname=="robot-learning":
            shortname="CoRL"
        
        elif shortname=="cclear":
            shortname="CLeaR"
        
        elif shortname=="thecvf":
            shortname="ECCV"
        
        elif shortname=="graphicsinterface":
            shortname="GI"
        
        elif shortname=="iscaconf":
            shortname="ISCA"
        
        elif shortname=="auai":
            shortname="UAI"
        
        elif shortname=="roboticsfoundation":
            shortname="RSS"
            
        rank = find_rank(conf_rank, shortname)
        set_conf_rank(conf["id"], rank, shortname)



In [120]:
add_rank_to_conferences()

ICLR
A*
ICLR
A*
ICLR
A*
ICLR
A*
ICLR
A*
ICLR
A*
ICML
A*
ICML
A*
ICML
A*
ESWC
B
ESWC
B
MIDL
Unranked
RSS
A*
RSS
A*
vivoconference
Unranked
MICCAI
A
MICCAI
A
NeurIPS
A*
NeurIPS
A*
NeurIPS
A*
NeurIPS
A*
ACL
A*
ACL
A*
HRI
A
HRI
A
IJCAI
A*
kg-construct
Unranked
GeoMedIA
Unranked
UAI
A
ACM
Unranked
ACM
Unranked
ACM
Unranked
ACM
Unranked
AKBC
Unranked
AKBC
Unranked
AKBC
Unranked
ICAPS
A*
ICAPS
A*
ICAPS
A*
ICAPS
A*
ICAPS
A*
ECCV
A*
ECCV
A*
GI
C
GI
C
GI
C
Mobisys
A
ECML PKDD
A
tinyml
Unranked
ML_Reproducibility_Challenge
Unranked
ML_Reproducibility_Challenge
Unranked
CoRL
Unranked
CoRL
Unranked
CoRL
Unranked
ECML PKDD
A
CLeaR
Unranked
uoft
Unranked
AAAI
A*
AAAI
A*
WBIR
Unranked
automl
Unranked
automl
Unranked
ISCA
A*
logconference
Unranked
JSYS
Unranked
JSYS
Unranked
JSYS
Unranked
IEEE
Unranked
Interspeech
A
NoDaLiDa
Unranked
RBCDSAI
Unranked


In [17]:
from dotenv import load_dotenv
import openreview
import os
load_dotenv("credentials.env")
client = openreview.api.OpenReviewClient(
    baseurl = 'https://api2.openreview.net',
    username = os.getenv("OR_USERNAME"),
    password = os.getenv("OR_PASSWORD"),
)
venue = "ICML.cc/2023/Conference"
venue_group = client.get_group(venue)
if venue_group.content is not None:
    print(venue_group.content)
    submission_name = venue_group.content['submission_id']['value']
    review_name = venue_group.content['review_name']['value']
    all_subs = client.get_all_notes(invitation=f'{submission_name}')

{'submission_id': {'value': 'ICML.cc/2023/Conference/-/Submission'}, 'meta_invitation_id': {'value': 'ICML.cc/2023/Conference/-/Edit'}, 'submission_name': {'value': 'Submission'}, 'submission_venue_id': {'value': 'ICML.cc/2023/Conference/Submission'}, 'withdrawn_venue_id': {'value': 'ICML.cc/2023/Conference/Withdrawn_Submission'}, 'desk_rejected_venue_id': {'value': 'ICML.cc/2023/Conference/Desk_Rejected_Submission'}, 'rejected_venue_id': {'value': 'ICML.cc/2023/Conference/Rejected_Submission'}, 'public_submissions': {'value': False}, 'public_withdrawn_submissions': {'value': False}, 'public_desk_rejected_submissions': {'value': False}, 'title': {'value': 'Fortieth International Conference on Machine Learning'}, 'subtitle': {'value': 'ICML 2023'}, 'website': {'value': 'https://icml.cc/Conferences/2023'}, 'contact': {'value': 'program-chairs@icml.cc'}, 'program_chairs_id': {'value': 'ICML.cc/2023/Conference/Program_Chairs'}, 'reviewers_id': {'value': 'ICML.cc/2023/Conference/Reviewers'}

Getting V2 Notes: 100%|█████████▉| 1826/1828 [00:00<00:00, 1852.43it/s]


In [22]:
all_subs[0]

Note(id = 'zzPMd0Ue4i',number = 4787,cdate = 1674747836694,pdate = 1682373314831,odate = 1686841478827,mdate = 1686841478841,tcdate = 1674747836694,tmdate = 1686841478841,ddate = None,content = {'title': {'value': 'Differentially Private Sharpness-Aware Training'}, 'authors': {'value': ['Jinseong Park', 'Hoki Kim', 'Yujin Choi', 'Jaewook Lee']}, 'authorids': {'value': ['~Jinseong_Park1', '~Hoki_Kim1', '~Yujin_Choi1', '~Jaewook_Lee1']}, 'abstract': {'value': 'Training deep learning models with differential privacy (DP) results in a degradation of performance. The training dynamics of models with DP show a significant difference from standard training, whereas understanding the geometric properties of private learning remains largely unexplored. In this paper, we investigate sharpness, a key factor in achieving better generalization, in private learning. We show that flat minima can help reduce the negative effects of per-example gradient clipping and the addition of Gaussian noise. We t

In [19]:
from get_data_from_or import get_reviews

In [20]:
get_reviews(all_subs[0])

TypeError: 'NoneType' object is not subscriptable

In [21]:
for sub in all_subs:
    if sub.details is not None:
        print(sub.details)